## Medical Chatbot

# Warning: Obsolete

In [1]:
import os
os.chdir("../")

%pwd

'd:\\Medical-chatbot'

In [15]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore

from pinecone import Pinecone
from pinecone import ServerlessSpec

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate


### Load env

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

### Extract data from pdf

In [4]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [17]:
extracted_data = load_pdf_file("data/")


In [ ]:
extracted_data

### Chunking of data

In [19]:
def text_split(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

In [20]:
text_chunks = text_split(extracted_data)

In [ ]:
print(len(text_chunks))

### Download Embedding Model

In [7]:
def download_embedding_model():
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding_model


In [8]:
embedding_model = download_embedding_model()


### Pinecode - Vector Database

In [4]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))


In [ ]:
index_name = "medical-chatbot"
dimension_of_vector = 384

pc.create_index(
    name=index_name,
    dimension=dimension_of_vector,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [ ]:
# Embed and upsert the embeddings into the Pinecone Index

doc_store = PineconeVectorStore.from_documents(
    index_name=index_name,
    embedding=embedding_model,
    documents=text_chunks
)


In [9]:
# Load Existing Index

doc_store = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding_model
)


### Retrieval from database

In [40]:
retriever = doc_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [50]:
retrieved_docs = retriever.invoke("Can you suggest proper medication for early diagnosis of diabetes?")


In [51]:
retrieved_docs

[Document(id='bbbee54d-99e3-46c6-8038-86bf66c6b8ef', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2016-02-07T11:23:03+07:00', 'page': 1188.0, 'page_label': '1159', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'data\\encyclopedia-of-medicine-vol-1-5-3rd-edition.pdf', 'total_pages': 4505.0}, page_content='symptoms of diabetes and supporting patients with\nthe disease.Acupuncture can help relieve the pain\nassociated withdiabetic neuropathyby stimulation of\ncetain points. A qualified practitioner should be con-\nsulted. Herbal remedies also may be helpful in manag-\ning diabetes. Although there is no herbal substitute\nfor insulin, some herbs may help adjust blood sugar\nlevels or manage other diabetic symptoms. Some\noptions include:\n/C15fenugreek (Trigonella foenum-graecum ) has been'),
 Document(id='00cf28a2-6030-4fa8-a171-4f1d7b4106e4', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'm

### LLM Model

In [52]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [53]:
system_prompt = (
    "You are a medical expert. Answer the user's question based on the provided context."
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Can you suggest proper medication for early diagnosis of diabetes?")
    ]
)

In [54]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [55]:
response = rag_chain.invoke({"input": "Can you suggest proper medication for early diagnosis of diabetes?"})
print(response)

{'input': 'Can you suggest proper medication for early diagnosis of diabetes?', 'context': [Document(id='bbbee54d-99e3-46c6-8038-86bf66c6b8ef', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2016-02-07T11:23:03+07:00', 'page': 1188.0, 'page_label': '1159', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'data\\encyclopedia-of-medicine-vol-1-5-3rd-edition.pdf', 'total_pages': 4505.0}, page_content='symptoms of diabetes and supporting patients with\nthe disease.Acupuncture can help relieve the pain\nassociated withdiabetic neuropathyby stimulation of\ncetain points. A qualified practitioner should be con-\nsulted. Herbal remedies also may be helpful in manag-\ning diabetes. Although there is no herbal substitute\nfor insulin, some herbs may help adjust blood sugar\nlevels or manage other diabetic symptoms. Some\noptions include:\n/C15fenugreek (Trigonella foenum-graecum ) has been'), Document(id='00cf28a2-6030-4fa8-a171-4f1d7b4106e4',

In [56]:
print(response["answer"])


There is no medication for the *early diagnosis* of diabetes.  Diagnosis of diabetes relies on tests like fasting blood glucose, HbA1c, and oral glucose tolerance tests, not medication.  Medication is used to *treat* diabetes once it's diagnosed.
